In [ ]:
import openmc
import numpy as np
import matplotlib.pyplot as plt


#import the Cross sections file 
openmc.config['cross_sections'] = "/home/f_z/endfb-vii.1-hdf5/cross_sections.xml"

In [ ]:
###Fuel Pellets###

###Uranium###
uo2=openmc.Material(material_id=1)
uo2.add_element('U',1,enrichment=5.0) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2.add_element('O',2)
uo2.set_density('g/cm3',10.3)

###Plutonium###
'''
puo2=openmc.Material(material_id=2)
puo2.add_nuclide('Pu239',.94)#automatically it knows that the enrihment is 5%
puo2.add_nuclide('Pu240',.05) 
puo2.add_nuclide('Pu241',.01) 
puo2.add_element('O',2)
puo2.set_density('g/cm3',11.5)

###Mox###
mox=openmc.Material.mix_materials([uo2,puo2],[0.85,0.15],'wo')
#print(uo2,puo2,mox)
'''

In [ ]:
###Coolant###

water=openmc.Material(material_id=2)
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3',0.650)

###Assembly###

helium=openmc.Material(material_id=3)
helium.add_element('He',1)
helium.set_density('g/cm3',0.0043)


###Cladding### (from Input Correlations for Irradiation Creep of FeCrAl and SiC Based...)
#and Analysis of Options andExperimental Examination of Fuels for Water Cooled Reactors with Increased Accident Tolerance (ACTOF)

zircaloy=openmc.Material(material_id=4)
zircaloy.add_element('Zr',0.9843,'wo')
zircaloy.add_element('Sn',0.0120,'wo')
zircaloy.add_element('Fe',0.0240,'wo')
zircaloy.add_element('Cr',0.0120,'wo')
zircaloy.set_density('g/cm3',6.57)
#print(zircaloy)

#plugs
steel= openmc.Material(material_id=5)
steel.add_element('C',0.0005,'wo')
steel.add_element('Ti',0.005,'wo')
steel.add_element('Cr',0.1625,'wo')
steel.add_element('Mn',0.02,'wo')
steel.add_element('Fe',0.65,'wo')
steel.add_element('Ni',0.14,'wo')
steel.add_element('Mo',0.022,'wo')
steel.set_density('g/cm3',7.55)

In [ ]:
materials=openmc.Materials([uo2,helium,water,zircaloy,steel])
materials.export_to_xml('materials_zr.xml')

In [ ]:
###Geometry of the single pellet###

#lateral surface, infinite cylinder centered in z axis, outer surfaces
frod_height=385.866  
r_p=0.857/2
r_g=r_p+0.016
r_c=r_g+0.0385
print(2*r_c)
s_pellet=openmc.ZCylinder(0,0,r_p)
s_gap=openmc.ZCylinder(0,0,r_g)
s_clad=openmc.ZCylinder(0,0,r_c)

#top and bottom surface
#-rodheight/2(b_bottom_plug)#bottom_plug_pos#pellet_stack#bottom_plenum_pos#top_plenum_pos(t_bottom_plug)#rodheight/2(t_top_plug)#

s_top=openmc.ZPlane(frod_height/2., boundary_type='reflective') #reflective measn that we don't have leakages
s_bottom=openmc.ZPlane(-frod_height/2, boundary_type='reflective') 

###full rod geometry, plenum only on the upper part of the rod
bottom_plug_pos=-frod_height/2+1.465
bottom_plenum_pos=frod_height/2-16.457-1.468
top_plenum_pos=frod_height/2-1.468

s_bottom_plenum= openmc.ZPlane(bottom_plenum_pos) 
s_top_plenum= openmc.ZPlane(top_plenum_pos)
s_b_top_end_plug= openmc.ZPlane(bottom_plug_pos) #surface between bottom plug and fuel stack
#s_b_bottom_end_plug=s_bottom

##t_top/t_bottom indicates the top plug (_top part or _bottom part)

#s_t_top_end_plug= s_top
#s_t_bottom_end_plug= s_top_plenum
#over the pellets       

#let's create a geometry to run the code:
rod_pitch=1.26
#edge = rod_pitch/2 * 1/np.sin(np.pi/3)
#hex_cell = openmc.model.SPrism(edge_length=edge,orientation='x')#,boundary_type='reflective')
square_cell = openmc.model.RectangularPrism(rod_pitch,rod_pitch,boundary_type='reflective')



In [ ]:
###Cell construction ###

c_pellet=openmc.Cell(name="Pellet")
c_pellet.region=+s_b_top_end_plug & -s_bottom_plenum & -s_pellet 
c_pellet.fill=uo2  

c_gap=openmc.Cell(name="Gap") #gas in the gap along the fuel stack
c_gap.region=+s_pellet & +s_b_top_end_plug & -s_bottom_plenum & -s_gap 
c_gap.fill=helium

# gas plenum
c_gas= openmc.Cell(name='gas plenum')
c_gas.region=  +s_bottom_plenum & - s_top_plenum & -s_gap
c_gas.fill= helium

c_clad=openmc.Cell(name="Clad")
c_clad.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad.fill=zircaloy

### Coolant cell ###
c_coolant=openmc.Cell(name="Coolant")
c_coolant.region= +s_bottom & -s_top & +s_clad & -square_cell# define a hexagonal region for the coolant
c_coolant.fill=water

#bottom plug
c_b_plug= openmc.Cell(name='Bottom plug')
c_b_plug.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_b_plug.fill= steel

#top plug
c_t_plug= openmc.Cell(name='top plug')
c_t_plug.region=  +s_top_plenum  & -s_top & -s_gap
c_t_plug.fill= steel

fuel_rod_universe=openmc.Universe(cells=[c_pellet,c_gap,c_gas,c_clad,c_coolant,c_t_plug,c_b_plug])

#we now define instead the pellet stack in the fuel cell
'''
pellet_stack =openmc.RectLattice (name='pellet stack')
pellet_stack.pitch= (2*r_c,2*r_c,) #size along the three direction
#we define th most negative corner of the lattice (like a parallelepipedo)
pellet_stack.lower_left= [-2*r_c,-2*r_c,bottom_plug_pos] 
#pellet_stack.upper_right=[-0.5,-0.5,bottom_plenum_pos]
pellet_stack.universes=fuel_rod_universe
#pellet stack
c_pellet_stack= openmc.Cell(name='Stack cell')
c_pellet_stack.region= +s_b_top_end_plug & -s_bottom_plenum
c_pellet_stack.fill= pellet_stack
'''

In [ ]:
fuel_rod_universe.plot(basis="xz",width=(1.5,390.9),color_by= 'material') #infinitely defined lateraly
fuel_rod_universe.plot(basis="xy",width=(1.5,1.5),color_by= 'material')
fuel_rod_universe.plot(basis="yz",width=(1.5,1.5),color_by= 'material') 

In [ ]:
### import the geometry in an xml file

fuel_cell_geom=openmc.Geometry(fuel_rod_universe) #before we didn't define the class
fuel_cell_geom.export_to_xml('geometry_zr.xml')


In [ ]:
#rod universe
'''
inclad_universe= openmc.Universe(cells=[c_pellet,c_gas,c_gap,c_clad,c_coolant,
                                       c_b_plug,c_t_plug])
inclad_universe.plot(basis='xz', width=(1.,385.15), color_by='material')
#
inclad= openmc.Cell(name='Components inside the cladding')
inclad.region= - s_t_top_end_plug & +s_b_bottom_end_plug &-s_gap
inclad.fill= inclad_universe

#clad, already defined c_clad 

#coolant, already defined c_coolant

fuel_rod_universe=openmc.Universe(cells=[inclad,c_clad,c_coolant]) 
'''

In [ ]:
###Tallies###

###Filters###
#for our purpose we need to use an energy filter

e_filter=openmc.EnergyFilter([.0,68e-2,20e6]) #2 energy groups, but for a fast reactor there are not so muche thermal ns

#meshes, where the neutrons are flying?
msh_filter=openmc.RegularMesh()#mesh_id=1) let's construct the grid, with number of nodes(dimensions)
msh_filter.dimension=[25,25,1] #x,y,z meshes generated in the different directions.this is a box,not an hexagon
msh_filter.lower_left=[-rod_pitch/2,-rod_pitch/2,-frod_height/2] #this are the dimensions of the meshes!
msh_filter.upper_right=[rod_pitch/2,rod_pitch/2,frod_height/2]
mesh_filter=openmc.MeshFilter(msh_filter)#filter_id=2)

###Scores###

#fluxes, of the whole region 
flux=openmc.Tally(name="Flux")
flux.scores=["flux"]
flux.filters=[e_filter,mesh_filter]


#fission reaction rate , to be recalled when you need to plot only the rod behavior (fission rate, indeed)
fission_rr=openmc.Tally(name="fission_rate")
fission_rr.scores=["fission"]
fission_rr.filters=[e_filter,mesh_filter]

In [ ]:
###exporting the tallies in a xml file

tallies=openmc.Tallies([flux,fission_rr])
tallies.export_to_xml()


In [ ]:
###run the simulation###

#Sources definitiion
source_point= openmc.stats.Point(xyz=(0.,0.,0.)) #where it starts the gen, or we can use a region
source_region=openmc.stats.Box((-r_p/2,-r_p/2,-frod_height/2),(r_p/2,r_p/2,frod_height/2)) #radius of the cladding, edges of the box
source_energy=openmc.stats.Watt()#by default the constrains define the rejection: 
                                 # fissionable means that it search onlyfor fissionable material
source=openmc.IndependentSource(space=source_region,energy=source_energy,constraints={"fissionable":True})
source_2=openmc.IndependentSource(space=source_point,energy=source_energy,constraints={"fissionable":False})

#simulation settings

sim_sett=openmc.Settings()
sim_sett.run_mode="eigenvalue"
sim_sett.particles=10000  #produced by the source, every time the batches rinormalize respect this number
                         #same number of neutrons per generation, every neutrons has a weight, 
                         # that means that not all the ns contribute to the
                         # in this example we are focusing on a NON criticality problem=> prim o poi i neutroni finiscono.
sim_sett.inactive=0
sim_sett.batches=10
sim_sett.source=source

sim_sett.export_to_xml()


In [ ]:
'''for cell in fuel_cell_geom.get_all_cells().values():
    print(f"Cell ID: {cell.id}, Region: {cell.region}")
#the output provide a dictionary with the list of different cell that are used to create the 
#universe (in this case fuel_rod_universe). The numbers inside parenthesis rappresents the surfaces
#that composed each cell, with negative and positive def based on what you did in the definition 
#of each region

# Get all surfaces from your geometry
surfaces = fuel_cell_geom.get_all_surfaces()

# Retrieve Surface 8
surface_8 = surfaces[8]  # Assuming 8 is a valid surface ID
surface_7=surfaces[7]
# Print surface properties (to understand what it is)
print(surface_8,surface_7)
'''
openmc.run()

In [ ]:
###How to verify that the sources are working, use of the h5 file
import glob
#function to search for the right statepoint file
def get_latest_statepoint():
    files = sorted(glob.glob("statepoint.*.h5"))
    return files[-1] if files else None  # Return None if no file exists

latest_file = get_latest_statepoint()
if latest_file:
    sp_file = openmc.StatePoint(latest_file)
    print(f"Using statepoint: {latest_file}")
else:
    print("No statepoint file found!")

#sp_file=openmc.StatePoint("statepoint.100.h5") #to be changed every time you re-run the code

'''
plt.quiver(sp_file.source["r"]["x"],sp_file.source["r"]["y"],sp_file.source["u"]["x"],sp_file.source["u"]["y"],
           np.log(sp_file.source["E"]),cmap="jet",scale=15.)#help to visualize the population and directions from the source 

plt.colorbar()
plt.ylim(-1.,1.)
plt.xlim(-1.,1.)

flux_tally = sp_file.get_tally(scores=['fission'], name='fission_rate') #instead you can do the same for the flux
flux_data=flux_tally.get_reshaped_data()

print(flux_tally.shape,flux_data.shape)
plt.imshow(flux_data[:, :, 0], extent=[-rod_pitch/2, rod_pitch/2, -edge, edge], origin='lower', cmap='inferno')
plt.colorbar(label='Neutron Flux')
plt.xlabel('X Position (cm)')
plt.ylabel('Y Position (cm)')
plt.title('Neutron Flux in X-Y Plane at Z = 10 cm')
plt.show()
#K_eff plot
plt.plot(sp_file.k_generation)
'''

###tally extraction:sp_file.get_tally(scores=["fission"],name=fission_rr) you need the name of the tally
#divide for each energy groups: use the energy filter "get_slicer function", use print(.shape to see the dimensions of the arrays)
#reshape the array of the sliced array(the square of the original mesh) using the function "mean.shape =(25,25)"
fr_tally = sp_file.get_tally(scores=['fission'], name='fission_rate') #instead you can do the same for the flux
#fr_tally = sp_file.get_tally(scores=['flux'], name='Flux') #instead you can do the same for the flux

print(fr_tally.shape)

fr1 = fr_tally.get_slice(filters=[openmc.EnergyFilter], filter_bins=[((0, 0.68),)])
fr2 = fr_tally.get_slice(filters=[openmc.EnergyFilter], filter_bins=[((0.68, 20e6),)])
# Correcting the shape mismatch
#fr1.mean = fr1.mean.sum(axis=-1).reshape(25, 25)  # Summing over z bins
#fr2.mean = fr2.mean.sum(axis=-1).reshape(25, 25)
fr1.mean.shape = (25, 25)
fr2.mean.shape = (25, 25)

plt.rcParams['figure.figsize'] = [14, 10]
fig = plt.subplot(121)
fig1 = plt.imshow(fr1.mean, label="E < 1 keV")
fig2 = plt.subplot(122)
fig2.imshow(fr2.mean, label="E > 800 keV")
